In [224]:
import pandas as pd
import gensim 
from gensim.models import Word2Vec 
from tqdm import tqdm
import re
import numpy as np
  
dataset = pd.read_csv("data/names_combined.csv")

dataset.dropna(inplace=True)
dataset.replace('b','black',inplace=True)
  
data = [] 
corpus = []

def features(name):
    name = re.sub(' ','',name)
    bigrams = []
    trigrams = []
    for i in range(len(name)-1):
        bigrams.append(name[i:i+2])
        if i < len(name) - 2:
            trigrams.append(name[i:i+3])
    trigrams.append(name[-3:])
    fets = bigrams+trigrams
    fets.append(name.split(' ')[0])
    fets.append(name.split(' ')[-1])
    return bigrams+trigrams

for i in tqdm(range(len(dataset['name']))):
    temp = []
    review = re.sub('[^a-zA-Z]', ' ', dataset['name'].values[i])
    review = re.sub('  ', ' ', review)
    review = review.lower()
    if review[0] == ' ':
        review=review[1:]
    temp = list(review) + features(review)
    data.append(temp)
    corpus.append(review)

100%|██████████| 120719/120719 [00:03<00:00, 33471.98it/s]


In [225]:
# Create & train Char vec model
model1 = gensim.models.Word2Vec(data, min_count = 1, size = 300, window = 10) ## 300,10
print('model created')
model1.train(data,total_examples=model1.corpus_count, epochs=15)
print('model trained')

model created
model trained


In [226]:
y = dataset.iloc[:, 2].values
X = []
for c in tqdm(corpus):
	vecs = model1[list(c) + features(c)]
	X.append(vecs.sum(axis=0))
X = np.array(X)

  0%|          | 0/120719 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
100%|██████████| 120719/120719 [00:26<00:00, 4633.33it/s]


In [227]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
ros = RandomOverSampler(random_state=42)
X, y = ros.fit_resample(X, y)

In [228]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(y.reshape(-1,1)).toarray()

In [229]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)#, random_state = 0)

In [230]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [231]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, LSTM, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers

classifier = Sequential()

######## race classification 87% test accuracy
classifier.add(Dense(input_dim=300, output_dim=256, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.2))  # Dropout overfitting
classifier.add(Dense(output_dim=256, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=128, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=128, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=64, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=32, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=4, activation='softmax'))

##### gender classification model 98% test acc
# classifier1.add(Dense(input_dim=300, output_dim=256, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))  # Dropout overfitting
# classifier1.add(Dense(output_dim=128, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=128, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=64, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=64, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=32, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=2, activation='softmax'))


classifier.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ['accuracy'])
history = classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=256)

# classifier1.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ['accuracy'])
# history1 = classifier1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=60, batch_size=256)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=300, activation="relu", units=256)`
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=256)`
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykern

Train on 155664 samples, validate on 38916 samples
Epoch 1/40
155664/155664 [==============================] - 20s 131us/step - loss: 0.6149 - acc: 0.7274 - val_loss: 0.4913 - val_acc: 0.7860
Epoch 2/40
155664/155664 [==============================] - 12s 78us/step - loss: 0.4940 - acc: 0.7847 - val_loss: 0.4491 - val_acc: 0.8040
Epoch 3/40
155664/155664 [==============================] - 15s 97us/step - loss: 0.4591 - acc: 0.7995 - val_loss: 0.4277 - val_acc: 0.8134
Epoch 4/40
155664/155664 [==============================] - 16s 102us/step - loss: 0.4366 - acc: 0.8100 - val_loss: 0.4110 - val_acc: 0.8207
Epoch 5/40
155664/155664 [==============================] - 12s 74us/step - loss: 0.4191 - acc: 0.8174 - val_loss: 0.3996 - val_acc: 0.8248
Epoch 6/40
155664/155664 [==============================] - 11s 74us/step - loss: 0.4047 - acc: 0.8234 - val_loss: 0.3916 - val_acc: 0.8287
Epoch 7/40
155664/155664 [==============================] - 14s 89us/step - loss: 0.3932 - acc: 0.8296 - va

In [232]:
def predict_gen(name):
    name = name.lower()
    classes = ['female','male']
    namevec = model1[list(name)+features(name)].sum(axis=0).flatten()
    pred = classifier1.predict(namevec.reshape(1,300))
    predprob = pred[0][np.argmax(pred)]*100
    gender = classes[np.argmax(pred)]
    return f'{name} is probably a {gender}.... chances:{predprob}%'

def predict_race(name):
    try:
        name = name.lower()
        classes = ['black','hispanic','indian','white']
        namevec = model1[list(name)+features(name)].sum(axis=0).flatten()
        pred = classifier.predict(sc.transform(namevec.reshape(1,300)))
        predprob = pred[0][np.argmax(pred)]*100
        race = classes[np.argmax(pred)]
    except:
        return "None"
    return f'{name} is {race}....{predprob}%.....probs:{pred[0]} for {classes} respectively'

In [240]:
predict_race('narendra modi')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


"narendra modi is indian....99.84652996063232%.....probs:[6.9197454e-04 4.4284065e-05 9.9846530e-01 7.9839397e-04] for ['black', 'hispanic', 'indian', 'white'] respectively"

In [234]:
y_pred = classifier.predict(X_test)

In [235]:
classes = ['black','hispanic','indian','white']
test_class = []
pred_class = []
for i in range(len(y_pred)):
    test_class.append([classes[np.argmax(y_test[i])]])
    pred_class.append([classes[np.argmax(y_pred[i])]])

In [236]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(test_class, pred_class)
print(cm)
print(classification_report(test_class, pred_class,target_names=classes))

[[7680   87   33 1967]
 [   0 9687    0    5]
 [  19    0 9595   25]
 [2404  894   77 6443]]
              precision    recall  f1-score   support

       black       0.76      0.79      0.77      9767
    hispanic       0.91      1.00      0.95      9692
      indian       0.99      1.00      0.99      9639
       white       0.76      0.66      0.71      9818

   micro avg       0.86      0.86      0.86     38916
   macro avg       0.86      0.86      0.86     38916
weighted avg       0.85      0.86      0.85     38916



In [ ]:
from keras.layers.embeddings import Embedding


In [241]:
y = dataset.iloc[:, 2].values
X = []
for c in tqdm(corpus):
	vecs = model1[list(c) + features(c)]
	X.append(vecs)
X = np.array(X)

  0%|          | 0/120719 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
100%|██████████| 120719/120719 [00:31<00:00, 3840.00it/s]


In [242]:
X.shape

(120719,)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten
from keras.layers.normalization import BatchNormalization

model = Sequential()
model.add(LSTM(300, return_sequences=True, input_shape=(30,28)))
model.add(Dropout(0.2))
model.add(LSTM(300, return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(output_dim=32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=2048)